># TensorFlow 준비하기

- TensorFlow는 Google이 2015년에 공개한 머신러닝 오픈소스 프레임워크임
- GCP와 상성이 매우 좋으며, GCS와 입출력을 연동하는 기능, Dataflow와 ML Engine 같은 TensorFlow를 풀 매니지드로 운용 가능한 환경 제공

In [11]:
import tensorflow as tf

___
>## Datalab에서 TensorFlow 사용하기

- TensorFlow를 활용한 예제는 아래를 참조:

In [1]:
with tf.Graph().as_default():
  # 상수 정의하기
  x = tf.constant(10.0, name = 'x')
  y = tf.constant(32.0, name = 'y')
  
  # 더하기
  op = tf.add(x, y)
  
  # 연산 실행하기
  with tf.Session() as sess:
    result = sess.run(op)

print(result)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


42.0


- TensorFlow의 흐름을 정리해 보면:  
    1. 계산 그래프 정의하기  
    2. 세션 정의하기  
    3. 연산 실행하기  
    순서로 프로세스 실행

#### 1. 계산 그래프 정의하기

- `tf.constant`로 상수를 정의 (`tf.placeholder`, `tf.variable` 등으로 계산그래프의 노드 정의 가능)
- `tf.add`로 연산을 정의하고 노드 간 관계를 정의
- 이렇게 만든 계산그래프를 `tf.Graph().as_default()`로 정의
- 단, with 구문으로 정의했으므로 구문을 벗어날 떄 자동 해제 (Datalab을 사용할 경우 특정 블록을 여러번 실행할 수 있으므로 with 구문으로 네트워크가 다른 이름으로 생성되는 것을 방지)

#### 2. 세션 정의하기

- `tf.Session()`으로 '계산을 실행하는 단위'인 세션을 정의

#### 3. 연산 실행하기
- 세션의 `run()`을 호출하여 계산그래프 계산을 실행

___
>## 변수와 플레이스홀더

- 실제 계산에서 x와 y에 상숫값이 설정되는 경우는 전무
- 변화하는 값을 다루는 경우를 지원하기 위해 '변수<sub>Variable</sub>'와 '플레이스홀더<sub>Placeholder</sub>'라는 기능 존재
- '변수'는 다양한 값을 넣을 수 있는 상자로 '가중치' 같이 학습에 따라 변화하는 값은 '변수로 정의하여 사용
- '플레이스홀더'는 외부에서 입력을 받아 변수로 사용하는 기능으로, 특징량을 입력받는 경우 플레이스홀더를 사용

In [10]:
with tf.Graph().as_default():
  # x는 플레이스홀더
  x = tf.placeholder(tf.int32, name = 'x')
  y = tf.constant(32, name = 'y')
  
  # 더하기
  op1 = tf.add(x,y)
  
  # z 변수 생성
  z = tf.Variable(0, name = 'z')
  
  # 연산 결과를 변수에 지정
  op2 = tf.assign(z, op1)
  
  # 변수 초기화하기
  init_op = tf.global_variables_initializer()
  
  # 연산 실행
  with tf.Session() as sess:
    sess.run(init_op)
    result = sess.run(op2, feed_dict = {x:10})


print(result)

42


- `y`는 상수값을 지정하였고, `x`는 `int32`를 자료형으로 갖는 플레이스홀더를 정의
- `sess.run()`의 매개변수로 `feed_dict = {x:10}`으로 플레이스홀더 값 전달
- 변수는 `tf.Variable(0, name = 'z')`로 초기값을 0으로 지정하였으나 `tf.global_variables_initializer()`로 상태 초기화(권장)
- 연산 결과를 변수 `z`에 대입할 때에는 `tf.assign`을 사용

___
>## 로지스틱 회귀로 식별하기